# How To: Compute likelihood and chi2

In [1]:
# Set up the path to SModelS installation folder
import sys; sys.path.append("."); import smodels_paths

In [2]:
from smodels.tools.simplifiedLikelihoods import LikelihoodComputer, Data

## How to compute the likelihood and chi2 from the number of events

In [3]:
# If the number of observed events, the number of expected background events,
# its error and the number of signal events and its error are known, the likelihood
# for the signal (assuming a truncated gaussian distribution for the background and signal uncertainties)
# can be computed as:
m=Data ( observed=5, backgrounds=4.2, covariance=0.71**2, third_moment=None, nsignal=.1 )
comp=LikelihoodComputer ( m )
print ('likelihood=',comp.likelihood(nsig = 10. ) )
print ('chi2=',comp.chi2( nsig=10.))

likelihood= 0.002043162995765441
chi2= 7.753011557803882


## How to compute the likelihood and chi2 from a theory prediction

In [4]:
# In most cases one wants to compute the likelihood and chi2 for a given theory prediction computed by SModelS.
# Below we generate theory predictions and compute the likelihood and chi2 values for them
# First we import those parts of smodels that are needed for this exercise
#(We will assume the input is a SLHA file. For LHE files, use the lheDecomposer instead)
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.theory import decomposer
from smodels.installation import installDirectory
from smodels.tools.physicsUnits import fb, GeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database

In [5]:
# Define the SLHA input file name
filename="inputFiles/slha/gluino_squarks.slha"
model = Model(BSMparticles = BSMList, SMparticles = SMList)
model.updateParticles(inputFile=filename)

In [6]:
# Load the database, do the decomposition and compute theory predictions:
# (Look at the theory predictions HowTo to learn how to compute theory predictions)
database = Database("official")
expResults = database.getExpResults()
topList = decomposer.decompose(model, sigmacut = 0.03 * fb, doCompress=True, doInvisible=True,minmassgap = 5* GeV)
allThPredictions = [theoryPredictionsFor(exp, topList) for exp in expResults]

In [7]:
# For each theory prediction, compute the corresponding likelihood and chi2 values
# (This is only possible for efficiency map-type results):
for i,thPreds in enumerate(allThPredictions):
    if not thPreds: continue #skip results with no predictions
    expID = expResults[i].globalInfo.id
    dataType = expResults[i].getValuesFor('dataType')[0]    
    for theoryPred in thPreds:
        #Compute the likelihood and chi2:
        theoryPred.computeStatistics()
        print ("\nExperimental Result: %s (%s-type)" %(expID,dataType)) #Result ID and type
        print ("Theory prediction xsec = ",theoryPred.xsection.value) #Signal xsection*efficiency*BR
        if dataType == 'efficiencyMap':
            theoryPred.computeStatistics()
            print('L_BSM, L_SM, L_max =', theoryPred.likelihood, theoryPred.lsm, theoryPred.lmax )

        else:
            print ("(likelihood not available)")



Experimental Result: ATLAS-SUSY-2015-06 (efficiencyMap-type)
Theory prediction xsec =  4.21E-03 [pb]
L_BSM, L_SM, L_max = 0.0004752701757045079 0.0005481440076086251 0.0005481440076086251

Experimental Result: ATLAS-SUSY-2016-07 (upperLimit-type)
Theory prediction xsec =  4.91E-02 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2016-07 (upperLimit-type)
Theory prediction xsec =  1.03E-01 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2016-07 (upperLimit-type)
Theory prediction xsec =  1.16E-02 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2016-07 (upperLimit-type)
Theory prediction xsec =  7.53E-03 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2016-07 (efficiencyMap-type)
Theory prediction xsec =  9.72E-03 [pb]
L_BSM, L_SM, L_max = 1.2303568763388563e-59 0.0009076683058591838 0.0009778159771421915

Experimental Result: ATLAS-SUSY-2016-14 (upperLimit-type)
Theory prediction xsec =  3.50E-05 [pb]
(likelihood not av


Experimental Result: ATLAS-SUSY-2013-11 (efficiencyMap-type)
Theory prediction xsec =  4.39E-05 [pb]
L_BSM, L_SM, L_max = 0.020796826168257933 0.01893458607388521 0.02164770708057637

Experimental Result: ATLAS-SUSY-2013-12 (upperLimit-type)
Theory prediction xsec =  1.85E-02 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2013-18 (upperLimit-type)
Theory prediction xsec =  3.07E-06 [pb]
(likelihood not available)

Experimental Result: ATLAS-SUSY-2013-18 (efficiencyMap-type)
Theory prediction xsec =  1.99E-07 [pb]
L_BSM, L_SM, L_max = 0.0004951111708257827 0.0004970970903613485 0.0004970970903613485

Experimental Result: ATLAS-SUSY-2013-20 (upperLimit-type)
Theory prediction xsec =  6.74E-03 [pb]
(likelihood not available)

Experimental Result: CMS-PAS-SUS-13-016 (upperLimit-type)
Theory prediction xsec =  3.07E-06 [pb]
(likelihood not available)

Experimental Result: CMS-PAS-SUS-13-016 (efficiencyMap-type)
Theory prediction xsec =  2.88E-08 [pb]
L_BSM, L_SM, L_max = 

In [8]:
## for an example that involves combination of signal regions and different treatment of the nuisances, 
## see the "marginalize" how to.